## 数据预处理

### 数据缺失情况

1. ff3(两次观测之间10到12米高度处的最大阵风): 3/4缺失
2. N(总云量): 1/5缺失
   - 补“无云”
3. WW(气象站报告的当前天气状况): 1/2缺失
   - 补“无法观测”
4. Tn/Tx(过去12小时内的最低/高温度): 大部分仅记录下/上午2时的数据
   - 线性插值
5. Nh(观测到的C1云层的数量): 1/4缺失
    - 补0
6. H(最低云层底部的高度): 1/2缺失
   - 补“无云”
7. Cl/Cm/Ch(层积云、雨层云和积雨云/高层云、高积云和高雨云/卷云、卷积云和卷层云): 3/4缺失
8. RRR/tR(降雨量/达到指定降雨量的时间): 大量缺失
    - 补0

In [12]:
import pandas as pd

# 删除不需要的特征
filename = 'training_datasets/training_dataset.csv'
output_filename = 'training_datasets/training_dataset_0.csv'
features = ['Time Stamp', 'T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3', 'N', 'WW', 'W', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'Td', 'tR', 'RRR']# 需要的特征

df = pd.read_csv(filename, delimiter=',', encoding='gb18030', low_memory=False)
df = df[features]

df.to_csv(output_filename)
print(df.isnull().sum())


Time Stamp        0
T                 3
Po              242
P               235
Pa            13735
U                 5
DD                0
Ff                0
ff10          40774
ff3           32882
N              8391
WW                0
W             26432
Tn            26703
Tx            27371
Cl            33023
Nh            20900
H             11298
Cm            32665
Ch            34002
VV             1115
Td                2
tR            26710
RRR           26710
dtype: int64


In [14]:
df.drop(['ff10'], axis=1, inplace=True)# 没有数据，直接删除特征
df.drop(['Tn', 'Tx'], axis=1, inplace=True)# 可以补值，暂时删除特征，以观察其余特征数据完整性
df.drop(['ff3', 'Cl', 'Cm', 'Ch'], axis=1, inplace=True)# 数据缺失过多，考虑删除特征

df.dropna(subset=['T', 'Po', 'P', 'U', 'VV', 'Td'], inplace=True)# 少量缺失，直接删除对应数据

df.reset_index(drop=True, inplace=True)
print(df.isnull().sum())

Time Stamp        0
T                 0
Po                0
P                 0
Pa            13473
U                 0
DD                0
Ff                0
N              8389
WW                0
W             25576
Nh            20082
H             10853
VV                0
Td                0
tR            25506
RRR           25506
dtype: int64


In [15]:
df['WW'].fillna('云量发展情况没有进行观测或无法观测。 ', inplace=True)
df['Nh'].fillna('0', inplace=True)
df['H'].fillna('2500或更高，或无云。', inplace=True)
df['RRR'].fillna('无降水', inplace=True)
df['tR'].fillna('0', inplace=True)

print(df.isnull().sum())

Time Stamp        0
T                 0
Po                0
P                 0
Pa            13473
U                 0
DD                0
Ff                0
N              8389
WW                0
W             25576
Nh                0
H                 0
VV                0
Td                0
tR                0
RRR               0
dtype: int64


In [16]:
df_complete = df.dropna()
print(df_complete.shape[0])

4964
